
   # Top 50 Anime Extraction from MyAnimeList API


#### Install Library necessary for the Project

In [1]:
# MySQL Connector
%pip install mysql-connector-python

# Pandas
%pip install pandas

# CSV
%pip install csv

# Json
%pip install json

# Requests
%pip install requests

# Python Dotenv
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# MySQL connector
import mysql.connector

# Pandas
import pandas as pd

# Os
import os

# Json
import json

# Requests
import requests

# Datetime
from datetime import datetime

# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv() 


True

#### Initialize the variables

In [3]:
# Variable to connect to the API
api_url = os.getenv("MYANIMELIST_URL")
api_key = os.getenv("MYANIMELIST_API_KEY")
api_hostname = os.getenv("MYANIMELIST_HOSTNAME")

# Variable to connect to MySQL Database
db_hostname = os.getenv("MYSQL_HOSTNAME")
db_user = os.getenv("MYSQL_USERNAME")
db_password = os.getenv("MYSQL_PASSWORD")
database = "TopAnime"
table_name = "Top50"

# file names
json_file = str(datetime.now().year) + "_" + str(datetime.now().month) + "_" + str(datetime.now().day) + "_" + "TopAnime.json"
csv_file = str(datetime.now().year) + "_" + str(datetime.now().month) + "_" + str(datetime.now().day) + "_" + "TopAnime.csv"
log_file = str(datetime.now().year) + "_" + str(datetime.now().month) + "_" + str(datetime.now().day) + "_" + "TopAnime_log.txt"

#### Create the necessary function

In [4]:
def write_to_file(filename, content, mode='a'):
    """Writes the given log to the log file.

    Args:
        filename (str): The name of the file to add the content.
        content (str): The content to write to the file.
        mode (str, optional): The file opening mode. Defaults to 'a' (append).
            Other modes include:
            - 'w' (write)
            - 'r+' (read and write)
    """

    try:
        with open(filename, mode, encoding='utf-8') as file:
            file.write(str(datetime.now()) + ": " + content + "\n")
    except IOError as e:
        print(f"Error writing to content to log file.': {e}")

#### Connect to the API and retrieve the data from the URL

In [5]:
#Define headers

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": api_hostname
}

In [6]:
# Make the GET request to myanimelistAPI
write_to_file(log_file, f"Extracting data from {api_url}","w")
try:
    response = requests.get(api_url, headers=headers)
    write_to_file(log_file, f"Extracting data from {api_url} successfully")
except:
    write_to_file(log_file, f"Extracting data from {api_url} failed")
    raise SystemExit("System stop after error of extracting data from the API")



In [7]:
# Convert the response to JSON
data = response.json() 

# Store the JSON data in a file
with open(json_file, "w") as file:
    json.dump(data, file)

write_to_file(log_file, f"Data saved to json file: {json_file}")    

Data Transformation

In [8]:
write_to_file(log_file, f"Starting transformation")

# Convert the data to a panda dataframe
df = pd.DataFrame(data)

In [9]:
# Drop the "picture_url" and "members" Columns
df.drop(["picture_url","members"],axis = 1,inplace=True)

In [10]:
# Extract number of episode from the columns type

df["Number_Episod"] = '' # add the columns "number eps" to the dataframe df

for row in range(len(df)):                          # for each row of the datafrane
    start = df.loc[row,"type"].find('(') + 1        # find the start position of the number of episode 
    end = df.loc[row,"type"].find(' eps')           # find the end position of the number of episode 
    try:
        nb_ep = int(df.loc[row,"type"][start:end])      # extract the number 
    except ValueError:
        nb_ep = None

    df.loc[row,"Number_Episod"] = nb_ep                # replace the default value inside the columns by the number of episode

In [11]:
# Extract the only the type from the columns type

for row in range(len(df)):                    # for each row of the datafrane
    end = df.loc[row,"type"].find(' (')       # find the end position of the number of episode 
    type_ = df.loc[row,"type"][0:end]         # extract the type 
    df.loc[row,"type"] = type_                # replace the value inside the columns by the type extracted


In [12]:
# Extract start date and end date from the columns type

for row in range(len(df)):                                           # for each row of the dataframe
    start_date = df.loc[row,"aired_on"].split("-")[0].strip()        # extract the start date from the "aired_on" column
    start_date = datetime.strptime(start_date,'%b %Y').date()        # Convert the string to date format
    try:
        end_date = df.loc[row,"aired_on"].split("-")[1].strip()          # extract the end date from the "aired_on" column  
        end_date = datetime.strptime(end_date,'%b %Y').date()            # Convert the string to date format
    except ValueError:
        end_date = datetime.now().strftime(("%Y-%m-%d"))
    df.loc[row,"Start_date"] = start_date                            # replace the default value inside the columns by the start date
    df.loc[row,"End_date"] = end_date                                # replace the default value inside the columns by the end date

In [13]:
# Drop columns "aired_on", rename some columns and set colums "Rank" as index
df.drop(["aired_on"],axis = 1,inplace=True)
df.rename(columns={"rank": "Ranking", "title": "Title", "myanimelist_url": "Url", "myanimelist_id": "Id", "score": "Score", "type": "Type" },inplace=True)

write_to_file(log_file, f"Transformation finished successfuly")

In [14]:
# Convert the dataframe to csv file
df.to_csv(csv_file)

write_to_file(log_file, f"Data saved into the csv file: {csv_file}")

In [15]:
write_to_file(log_file, f"Connecting to MySQL {database} database")

# Connect to the database
try:

    cnx = mysql.connector.connect(
        host=db_hostname,
        user=db_user,
        password=db_password,
        database=database,
        port=3306
    )

    write_to_file(log_file, f"Connected to {database} database")

    # Create a cursor object
    cursor = cnx.cursor()

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [16]:
write_to_file(log_file, f"Creating table {table_name} into {database} if doesn't exist")

# Create Table Top50 into MYSQL
try:

    # Create a cursor object
    cursor = cnx.cursor()

    # SQL statement to create the table
    sql = """
    CREATE TABLE IF NOT EXISTS Top50 (
        Ranking INT UNIQUE NOT NULL PRIMARY KEY,
        Title VARCHAR(255),
        URL VARCHAR(255),
        Id INT,
        Score DOUBLE,
        Type VARCHAR(255),
        Number_Episod INT,
        Start_Date DATE,
        End_Date DATE)
    """
    
    
    # Execute the SQL statement
    cursor.execute(sql)

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file, f"Table {table_name} created into {database} successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [17]:
write_to_file(log_file,f"Sending {len(df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(df)} rows to MySQL table '{table_name}'.")
except mysql.connector.Error as e:
    print(f"Error sending data to MySQL: {e}")
    write_to_file(log_file,f"Fail to sent {len(df)} rows to MySQL table '{table_name}'.")
finally:
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()

In [18]:
columns = ', '.join(df.columns)
columns

'Title, Url, Id, Ranking, Score, Type, Number_Episod, Start_date, End_date'